# Image Super Resolution - Final Model Training
Now that we have a set of hyperparameters chosen through the evolution algorithm, we are ready to train the final model with all available data and chosen hyperparameters.

In [1]:
from google.colab import files

import keras
from keras.layers import Conv2D, UpSampling2D, Input, Add
from keras.models import Model
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from numpy.random import randint
import numpy as np
import os

Using TensorFlow backend.


### Hyperparameters

In [0]:
# These are the hyperparameters chosen after the evolution algorithm
h = {'l1_parameter': 0.01, 
     'l2_parameter': 0.016, 
     'num_residual_blocks': 9, 
     'num_conv_blocks': 2, 
     'num_final_conv_blocks': 3, 
     'num_epochs': 100, 
     'batch_size': 16, 
     'num_filters': 64, 
     'learning_rate': 0.0009, 
     'beta_1': 0.9, 
     'beta_2': 0.999}
h.update({
	"optimizer" : keras.optimizers.Adam(lr=h['learning_rate'], beta_1=h['beta_1'], beta_2=h['beta_2'], amsgrad=False),
	'regularizer' : l1_l2(l1=h['l1_parameter'], l2=h['l2_parameter'])
})

### Helper Functions

In [0]:
# a residual block
def residual_block(input_layer, activation='relu', kernel_size=(3,3)):
	global h
	layer = input_layer
	for i in range(h['num_conv_blocks']):
		layer = Conv2D(h['num_filters'], kernel_size, padding='same', activation=activation, activity_regularizer=h['regularizer'])(layer)
	conv_1x1 = Conv2D(3, (1,1), padding='same')(layer)
	return Add()([conv_1x1, input_layer])

In [0]:
# final convolution blocks
def conv_block(input_layer, kernel_size=(3,3)):
	global h
	layer = input_layer
	for i in range(h['num_final_conv_blocks']):
		layer = Conv2D(h['num_filters'], kernel_size, padding='same', activation='relu')(layer)
	return layer

In [0]:
# upsamples 2x
def upsample(layer):
	return UpSampling2D(size=(2,2))(layer)


In [0]:
# builds model based on hyperparameter specs
def build_model(shape=(150,150,3)):
  global h
  input_layer = Input(shape)
  layer = input_layer
  for i in range(h['num_residual_blocks']):
    layer = residual_block(layer)
  layer = upsample(layer)
  layer = conv_block(layer)
  output_layer = Conv2D(3, (1,1), padding='same')(layer)

  return Model(inputs=input_layer, outputs=output_layer)

In [0]:
# helper function for getting image file names
def get_filenames(directory):
	for _,_,filenames in os.walk(directory):
		pass
	return filenames

In [0]:
# returns dataset in (x_train, y_train), (x_test, y_test) format
from tqdm import tqdm

def build_dataset(directory):

	filenames = get_filenames(directory)
	X = []
	Y = []

	for filename in tqdm(filenames):
		image = Image.open(directory + filename)
		image_large = np.array(image)
		image_small = np.array(image.resize((150,150)))
		Y.append(image_large)
		X.append(image_small)

	X = np.asarray(X)
	X = X.astype('float32')
	X /= 255
	Y = np.asarray(Y)
	Y = Y.astype('float32')
	Y /= 255

	return (X, Y)

In [0]:
""" PSNR metric definition """
from keras import backend as K

def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

### Setup Training

In [10]:
""" Load Google Drive """
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [14]:
""" Load dataset into memory """
directory = PATH_TO_DIRECTORY # this is defined locally
X, Y = build_dataset(directory)

100%|██████████| 1087/1087 [08:59<00:00,  2.02it/s]


In [15]:
import time
import contextlib
import json
import warnings  
warnings.filterwarnings("ignore", category=DeprecationWarning) # ignore ugly tensorflow deprecation warnings


""" Choose a metric """
# metric = ['accuracy']
metric = [PSNR]

""" Build, compile and fit """
model = build_model(shape=(None,None,3))
model.compile(loss='mae', optimizer=h['optimizer'], metrics=metric)
model.fit(X, Y, batch_size=h['batch_size'], epochs=h['num_epochs'], verbose=1)










Epoch 1/100





1087/1087 [==============================] - 67s 61ms/step - loss: 31588.0522 - PSNR: 16.2942
Epoch 2/100
1087/1087 [==============================] - 47s 44ms/step - loss: 10074.8899 - PSNR: 20.5816
Epoch 3/100
1087/1087 [==============================] - 49s 45ms/step - loss: 5706.2219 - PSNR: 22.6168
Epoch 4/100
1087/1087 [==============================] - 49s 45ms/step - loss: 1848.2731 - PSNR: 26.9881
Epoch 5/100
1087/1087 [==============================] - 49s 45ms/step - loss: 576.1148 - PSNR: 31.4370
Epoch 6/100
1087/1087 [==============================] - 49s 45ms/step - loss: 298.8187 - PSNR: 33.3192
Epoch 7/100
1087/1087 [==============================] - 49s 45ms/step - loss: 162.5912 - PSNR: 34.7492
Epoch 8/100
1087/1087 [==============================] - 49s 45ms/step - loss: 101.4014 - PSNR: 34.8086
Epoch 9/100
1087/1087 [==============================] - 49s 45ms/step - loss: 69.1852 - PSNR: 35.6089
Epoch 10/100
1087/1087 [=====================

In [16]:
""" Save model """
model.save('model.h5')

print("done")

done
